# 見積もり・データは一設計関連

In [80]:
# %pip install pandas
# %pip install altair
# %pip install tablate

---

## 論物変換

TODO:

## Buffer Layout

TODO:

## DMA Scheme

TODO:

---

## 誤り訂正

悩んだので ECC 内臓の上位モデルを参考に設計する

- w/o ECC (こちらを使用している) [KIOXIA_TC58NVG0S3HTA00](https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/506/KIOXIA_TC58NVG0S3HTA00_Rev2.00_E191001C.pdf)
- w/ ECC [KIOXIA_TC58BVG0S3HTA00](https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/506/KIOXIA_TC58BVG0S3HTA00_REV2.00_E20191001C.pdf)

### Datasheet確認

#### ECC & Sector definition for ECC

- 2KByte pageは 512byte * 4area + 16byte * 4area で使用している
- main + spare の 512+16=528byteごとに、 9bitの検出と8bitの訂正能力を備える
- ECC parity code は 2112-2175 (64byte) に格納するので、ここにはアクセスできないという注意書き

main+spareごとECCのペアになっている点、ECC Status Readの戻り値がArea事に分かれていること、最後のECC parity code格納から以下のように見える

| sector | main [byte] | spare [byte] | ECC parity [byte] |
| ------ | ----------- | ------------ | ----------------- |
| 1st    | 0-511       | 2048 - 2063  | 2112 - 2127       |
| 2nd    | 512 - 1023  | 2064 - 2079  | 2128 - 2143       |
| 3rd    | 1024 - 1535 | 2080 - 2095  | 2144 - 2159       |
| 4th    | 1536 - 2047 | 2096 - 2111  | 2160 - 2175       |

data = 528byte (4224bit), parity = 16byte (128bit) で同じ訂正を組むことにする...と思ったが、 `9bitの検出と8bitの訂正能力を備える` と言うのが気になる。
一般(?)にいうECCは、1bitの訂正と2bitの検出が限界だったように思う。
ECCではない独自方式なのか、もしくは最大8bit検出というのはmain/spareを更に8分割したうえで、それぞれ独立した領域でエラーが出たときの理論的な条件でのCorrectableなのか。

以下を読むとCypress社のNOR Flashでは32byteごとにECC Encodeを施していて、1bit訂正2bit検出を拡張ハミング符号で実装している文言がある。
(総括部分の保証が若干謎の言い回しな気もするが...) 大きな単位ではなく細かめに保護をかけてもよいのかもしれない。

[SECDEC ECCスキームが2ビットを超えるエラーを処理する方法 - KBA222012 - Community Translated (JA) - infineaon Developer Community](https://community.infineon.com/t5/%E3%83%8A%E3%83%AC%E3%83%83%E3%82%B8%E3%83%99%E3%83%BC%E3%82%B9%E3%82%A2%E3%83%BC%E3%83%86%E3%82%A3%E3%82%AF%E3%83%AB-KBA/SECDEC-ECC%E3%82%B9%E3%82%AD%E3%83%BC%E3%83%A0%E3%81%8C2%E3%83%93%E3%83%83%E3%83%88%E3%82%92%E8%B6%85%E3%81%88%E3%82%8B%E3%82%A8%E3%83%A9%E3%83%BC%E3%82%92%E5%87%A6%E7%90%86%E3%81%99%E3%82%8B%E6%96%B9%E6%B3%95-KBA222012-Community-Translated-JA/ta-p/254390)

(17) Reliability Guidance によると 8bit ECC for each 512byte 必要 (これは w/o ECC modelにも記載あり) とある。
512byte中にどの程度誤りが発生するかによるが、2bit化けると訂正不能というのはギリギリを攻めているので、更に細かい単位で保護してあげようということだろうか。
ECCの重ねがけ (32byte保護した範囲を更に大きな単位で保護する? かもしくはその逆で大きな範囲で保護したものを細分化する？) みたいなことも考えたが、ECCメモリ関連の情報をざっと調べる限りは見当たらないので今回はやめておく。

ついでに同じ商で見つけたが0 data paddingをやめろという記載もあり。このUSBメモリにはAll Zeroのバイナリを置かないでください、とは言えないので対策が必要だろう...。
場当たり的な対策も考えはしたが、高速シリアル伝送などでもDC balance取るようなエンコーディングは使われていたのでこれをもとに考えていた。
元データに対してテーブル置換で実装できる8b10b encodeとその符号化率を上げたものを今回は検討にいれることにした。実用性としてこれでいいのかは正直良くわからないが、All Zero dataにはならなそうなので良いとする。

気になる点が処理順である。ECC -> 8b/10b だと、復号化するときに化けたものをもとに8b/10b decodeして最初のエラーよりも大きな誤りが伝搬してしまうように考える。
そのため、元データ -> 8b/10bしたものを書き込む元データとして捉え、これをECC Encodeするのが良さそうだ。

生成したECC Parity codeの置き場所も気になる。先述の表だと2112-2175byteに束ねられているように推察できる。
仮に32byte単位でECC Encodeを施しているとしたら元データの直後に配置することも可能だが (e.g. data0, parity0, data1, parity1, ... )
おそらくユーザーが書いたところはオリジナルのままにしたいとか、都度Encode/Decode挟んだりデータシートにあるPartial Programのことを考えると無理だったのだろう。
今回どちらにするか悩んだが、データ反転が起きなければそのままデータが使えたほうがいいと思うので、これはw/ ECC modelを見習うことにする

以下Codeで見積もりした結果出力

|    |   dc_balance_encode_size.data_bits |   dc_balance_encode_size.total_bits |   ecc_encode_size.data_bits |   ecc_encode_size.parity_bits |   code_rate |   surplus_bytes |   max_ecc_error_detection_count |
|---:|-----------------------------------:|------------------------------------:|----------------------------:|------------------------------:|------------:|----------------:|--------------------------------:|
| 21 |                                 64 |                                  66 |                         502 |                             9 |    0.950758 |              22 |                               8 |
| 48 |                                128 |                                 130 |                         247 |                             8 |    0.95     |              21 |                              16 |
| 49 |                                128 |                                 130 |                         502 |                             9 |    0.965385 |              55 |                               8 |

`surplus_bytes` が格納データ+Encode冗長部を全体から差し引いた空きbyte。デバッグ用データなどを入れ込めそう
`max_ecc_error_detection_count` がECC Errorで検出可能な最大数（ただし全範囲独立して起きた場合）。若干見づらいので書き直したものが以下。

128b/130b EncodeのパフォーマンスやCode Sizeに問題がなければ、64b/66b Encodeは棄却して良さそうに見える。
ECCの保護単位を256byte,512byte (w/ parity) どちらで構成するかは、少ない単位で構成できていたほうが体制が上がりそう。一方で空きbyte数の余裕観点で管理データがどうしても21byteで収められないなら512byteで構成するしかない。


| | ECC                 | DC Balance | 自由に使える空きbyte | pageあたりのECC保護分割数    | 採択時のpros |
|-|---------------------|-----------:|--------------------:|----------------------------:|-------------|
|1|拡張Hamming(256, 247)| 128b/130b  | 21                  | 16                          |特別な理由がなければ採用。推奨ECC厚生よりも更に細分化して構成できているので、エラー多発時の訂正能力が高い|
|2|拡張Hamming(512, 502)| 128b/130b  | 55                  | 8                           |USB Hostからの書き込み以外の管理データを配置するケースで、これが21byte/22byte以上必要な場合|
|3|拡張Hamming(512, 502)| 64b/66b    | 22                  | 8                           |128b/130b encodeの実装・動作上の問題がある場合|

#### 空きbyteの使い方

TODO:



### DC Balance Encode + ECC Encode時の効率・余剰byte数計算

In [89]:
import os
import math
from dataclasses import asdict, dataclass, field

import altair as alt
import pandas as pd
from tabulate import tabulate


pd.set_option("display.max_columns", 256)
pd.set_option("display.max_rows", 256)


@dataclass(frozen=True)
class DcBalanceEncodeSize:
    """
    DC Balance Encodeの設定
    """

    data_bits: int
    total_bits: int

    @property
    def code_rate(self) -> float:
        return self.data_bits / self.total_bits


@dataclass(frozen=True)
class EccEncodeSize:
    """
    ECCの設定
    """

    parity_bits: int
    additional_bits: int
    data_bits: int

    @property
    def total_bits(self) -> int:
        return self.data_bits + self.parity_bits + self.additional_bits

    @property
    def code_rate(self) -> float:
        return self.data_bits / self.total_bits

    @classmethod
    def from_parity_bits(
        cls, parity_bits: int, additional_bits: int = 1
    ) -> "EccEncodeSize":
        """
        Create an ECC frame size from the number of parity bits and additional bits.
        :param parity_bits: The number of parity bits.
        :param additional_bits: The number of additional bits.
        :return: The ECC frame size.
        """
        data_bits = 2**parity_bits - parity_bits - 1
        return cls(parity_bits, additional_bits, data_bits)


@dataclass(frozen=False)
class EncodeConfig:
    """
    NAND ICのPage不揮発化におけるエンコード設定
    """

    ###############################################################
    # ユーザー設定

    # DC Balance Encodeの設定
    dc_balance_encode_size: DcBalanceEncodeSize

    # ECCの設定
    ecc_encode_size: EccEncodeSize

    ###############################################################
    # ユーザー設定可能だが、基本変更する必要はない

    # USB Hostからのデータ最小単位
    logical_data_bytes: int = 512

    # 1ページあたりにUSB Hostからのデータを何個格納するか
    logical_data_num_per_page: int = 4

    # 1pageあたりのデータ量
    bytes_per_page = 2048 + 128

    # ECC保護すべきデータサイズの最小単位
    require_ecc_protection_bytes: int = 512

    ###############################################################
    # __post_init__で計算する値

    # 1pageあたりのUSB Hostから書き込まれるデータサイズ
    logical_data_bytes_per_page: int = field(default=None)

    # 1pageあたりの冗長データ量
    redundant_bytes: int = field(default=None)

    # 符号化率
    code_rate: float = field(default=None)

    # logical_data_bytes_per_pageに対して、冗長データ量を割り当てたときの、合計のデータ量
    require_bytes: int = field(default=None)

    # 冗長データ量が足りるかどうか
    is_sufficient_total_bytes: int = field(default=None)

    # 冗長データで余るバイト数
    surplus_bytes: int = field(default=None)

    # 冗長データが足りないバイト数
    short_bytes: int = field(default=None)

    # ECC保護が十分かどうか
    is_sufficient_ecc_protection: int = field(default=None)

    # ECC保護で require_ecc_protection_bytes で指定された中で検出できるエラーの最大数. 保護範囲が複数に分かれる場合はそれぞれで発生する理想ケースを想定
    max_ecc_error_detection_count: int = field(default=None)

    def __post_init__(self):
        self.logical_data_bytes_per_page = (
            self.logical_data_bytes * self.logical_data_num_per_page
        )
        self.redundant_bytes = self.bytes_per_page - self.logical_data_bytes_per_page
        self.code_rate = (
            self.ecc_encode_size.code_rate * self.dc_balance_encode_size.code_rate
        )
        self.require_bytes = math.floor(
            self.logical_data_bytes_per_page * (1.0 / self.code_rate)
        )
        self.is_sufficient_total_bytes = self.require_bytes <= self.bytes_per_page
        self.surplus_bytes = (
            self.bytes_per_page - self.require_bytes
            if self.is_sufficient_total_bytes
            else 0
        )
        self.short_bytes = (
            self.require_bytes - self.bytes_per_page
            if not self.is_sufficient_total_bytes
            else 0
        )
        self.is_sufficient_ecc_protection = (
            self.ecc_encode_size.total_bits <= self.require_ecc_protection_bytes
        )
        self.max_ecc_error_detection_count = math.floor(
            self.require_ecc_protection_bytes * 8 / self.ecc_encode_size.total_bits
        )


############################################
# 見積もり
dc_balance_encode_size_list = [
    DcBalanceEncodeSize(8, 10),
    DcBalanceEncodeSize(64, 66),
    DcBalanceEncodeSize(64, 67),
    DcBalanceEncodeSize(128, 130),
    # DcBalanceEncodeSize(1, 1),  # disable
]

parity_len_n = 16  # 計算対象にするparity長
ecc_encode_size_list = [
    EccEncodeSize.from_parity_bits(i) for i in range(2, parity_len_n)
]

encode_config_list = [
    EncodeConfig(dc_balance_encode_size=dc, ecc_encode_size=ecc)
    for dc in dc_balance_encode_size_list
    for ecc in ecc_encode_size_list
]
encode_config_df = pd.json_normalize(([asdict(c) for c in encode_config_list]))


def print_chart(df: pd.DataFrame):
    base_chart_1 = (
        alt.Chart(df)
        .mark_point()
        .encode(
            alt.X("ecc_encode_size\\.data_bits:N"),
            alt.Row("dc_balance_encode_size\\.data_bits:N"),
        )
        .properties(width=300, height=100)
    )
    base_chart_2 = (
        alt.Chart(df)
        .mark_rect()
        .encode(
            alt.X("ecc_encode_size\\.data_bits:N"),
            alt.Y("dc_balance_encode_size\\.data_bits:N"),
        )
        .properties(width=300, height=300)
    )
    alt.vconcat(
        # 横軸: ECCのデータビット数, 縦軸: 冗長データ量, 行: DC Balance Encodeのデータビット数
        alt.hconcat(
            base_chart_1.encode(
                alt.Y("surplus_bytes:Q"),
                alt.Color("is_sufficient_total_bytes:N"),
            ),
            base_chart_1.encode(
                alt.Y("code_rate:Q"),
                alt.Color("is_sufficient_ecc_protection:N"),
            ),
        ).resolve_scale(color="independent"),
        # Encode設定と余裕度のヒートマップ
        alt.hconcat(
            base_chart_2.encode(
                alt.Color("surplus_bytes:Q"),
            ),
            base_chart_2.encode(
                alt.Color("code_rate:Q"),
            ),
        ).resolve_scale(color="independent"),
    ).resolve_scale(color="independent").display()


# 今回の要件を満たせるもの
available_encode_config_df = encode_config_df.query(
    "is_sufficient_total_bytes and is_sufficient_ecc_protection"
)
print(
    tabulate(
        available_encode_config_df[
            [
                "dc_balance_encode_size.data_bits",
                "dc_balance_encode_size.total_bits",
                "ecc_encode_size.data_bits",
                "ecc_encode_size.parity_bits",
                # "ecc_encode_size.additional_bits",
                "code_rate",
                "surplus_bytes",
                "max_ecc_error_detection_count",
            ]
        ],
        headers="keys",
        tablefmt="pipe",
    )
)

# 全体
print_chart(encode_config_df)
print_chart(available_encode_config_df)
available_encode_config_df


|    |   dc_balance_encode_size.data_bits |   dc_balance_encode_size.total_bits |   ecc_encode_size.data_bits |   ecc_encode_size.parity_bits |   code_rate |   surplus_bytes |   max_ecc_error_detection_count |
|---:|-----------------------------------:|------------------------------------:|----------------------------:|------------------------------:|------------:|----------------:|--------------------------------:|
| 21 |                                 64 |                                  66 |                         502 |                             9 |    0.950758 |              22 |                               8 |
| 48 |                                128 |                                 130 |                         247 |                             8 |    0.95     |              21 |                              16 |
| 49 |                                128 |                                 130 |                         502 |                             9 |    0.965385 |   

alt.VConcatChart(...)

alt.VConcatChart(...)

,logical_data_bytes,logical_data_num_per_page,require_ecc_protection_bytes,logical_data_bytes_per_page,redundant_bytes,code_rate,require_bytes,is_sufficient_total_bytes,surplus_bytes,short_bytes,is_sufficient_ecc_protection,max_ecc_error_detection_count,dc_balance_encode_size.data_bits,dc_balance_encode_size.total_bits,ecc_encode_size.parity_bits,ecc_encode_size.additional_bits,ecc_encode_size.data_bits
21,512,4,512,2048,128,0.950758,2154,True,22,0,True,8,64,66,9,1,502
48,512,4,512,2048,128,0.950000,2155,True,21,0,True,16,128,130,8,1,247
49,512,4,512,2048,128,0.965385,2121,True,55,0,True,8,128,130,9,1,502
